In [1]:
import tensorflow as tf
import numpy as np

from pinn import PINNModel, model1, optm1
from data_gen import DataGenerator, real_u1, get_data

2024-10-19 15:22:51.070364: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 15:22:51.080799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 15:22:51.093534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 15:22:51.097260: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 15:22:51.107088: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [3]:
data = get_data(True)
EPOCHS = data['epochs']
koefs = data['koefs']
session = '3'

In [4]:
model = PINNModel(
    model1(), optm1(), lossf="mae", initial_weights=DataGenerator.init_name(session))

# so now there are weights at this path
model.save(DataGenerator.init_name(session))
dg = DataGenerator(data['x'], data['y'], model.predict, real_u1)

In [5]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
dataset = dg.inner_pairs(area_grid_size), dg.border_pairs(border_grid_size)

for koef in koefs:
    model = PINNModel(
        model1(), optm1(), "mae", initial_weights=DataGenerator.init_name(session))
    # model.reset()
    model.fit(koef, *dataset, EPOCHS)
    model.save(dg.name(koef, session))

epoch: 0 loss: 122.653351
epoch: 999 loss: 0.0122083984
Time past 5.683160781860352

epoch: 0 loss: 122.730392
epoch: 999 loss: 0.0132424701
Time past 4.699845552444458

epoch: 0 loss: 126.46701
epoch: 999 loss: 0.0115024243
Time past 4.2990007400512695

